Similar to the productivity analysis described above, similar results could be obtained by either applying a constrain to the flux of biomass production and squalene production, or define a correlation between the two by using quadratic objective function. The following section shows the process of applying constrains to ensure the ratio between biomass flux and squalene flux to be  

In [1]:
#Packages
import cobra
from cobra import Reaction, Metabolite, Model
from cobra.io import read_sbml_model
from cobra.flux_analysis import flux_variability_analysis

#Model input
modelQO=read_sbml_model('iJN678mod.xml')
co2QO=modelQO.reactions.get_by_id("EX_co2_e")
photonQO=modelQO.reactions.get_by_id("EX_photon_e")
hco3QO=modelQO.reactions.get_by_id("EX_hco3_e")

co2QO.bounds=(-3.7,1000.0)
hco3QO.bounds= (-3.7,1000.0)
photonQO.bounds= (-100,0)

medium = modelQO.medium
medium["EX_glc__D_e"] = 0.0
medium["EX_hco3_e"] = 0.0
medium["EX_photon_e"]=54.5
modelQO.medium = medium


In [2]:
modelQO.objective = "BIOMASS_Ec_SynAuto"
max_growth_rate = modelQO.optimize().objective_value
modelQO.objective = "EX_sql_e"
max_production_rate = modelQO.optimize().objective_value
print (max_growth_rate,max_production_rate)

0.08835330630568733 0.1233333333333334


In [3]:
ratio = max_production_rate/max_growth_rate
ratio

1.3959107869333285

In [4]:
same_flux = modelQO.problem.Constraint(
    modelQO.reactions.get_by_id('EX_sql_e').flux_expression - ratio* modelQO.reactions.get_by_id("BIOMASS_Ec_SynAuto").flux_expression,
    lb=0,
    ub=0)
modelQO.add_cons_vars(same_flux)

In [ ]:
#modelQO.solver = 'glpk'
#quadratic_objective = modelQO.problem.Objective(1 * modelQO.reactions.get_by_id('EX_sql_e').flux_expression + 1.3959 * modelQO.reactions.get_by_id("BIOMASS_Ec_SynAuto").flux_expression, direction='max')


In [6]:
modelQO.objective = "EX_sql_e"
modelQO.optimize().objective_value

0.06166666666666661

In [7]:
modelQO.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.0001993,0,0.00%
co2_e,EX_co2_e,3.7,1,100.00%
cobalt2_e,EX_cobalt2_e,0.0001437,0,0.00%
cu2_e,EX_cu2_e,0.0001329,0,0.00%
fe2_e,EX_fe2_e,0.0003307,0,0.00%
fe3_e,EX_fe3_e,0.0003011,0,0.00%
h2o_e,EX_h2o_e,2.751,0,0.00%
h_e,EX_h_e,0.5014,0,0.00%
k_e,EX_k_e,0.007475,0,0.00%
mg2_e,EX_mg2_e,0.001271,0,0.00%


In [8]:
print(modelQO.optimize().fluxes['BIOMASS_Ec_SynAuto'], modelQO.optimize().fluxes['EX_sql_e'])

0.044176653152843574 0.06166666666666658


In [12]:
reactions_QO = [modelQO.reactions.SQLS, modelQO.reactions.DXPS, modelQO.reactions.IPDDI,
     modelQO.reactions.GRTT, modelQO.reactions.BIOMASS_Ec_SynAuto]
fva_QO = flux_variability_analysis(modelQO, reaction_list = reactions_QO, fraction_of_optimum=1)
print (fva_QO)

                     minimum   maximum
SQLS                0.061667  0.061667
DXPS                0.377908  0.377908
IPDDI              -0.252597  0.125311
GRTT                0.125311  0.125311
BIOMASS_Ec_SynAuto  0.044177  0.044177


In [13]:
#To compare with model optimised for cell growth
model_growth=read_sbml_model('iJN678mod.xml')  
model_growth.objective = "BIOMASS_Ec_SynAuto"

In [14]:
reactions_growth = [model_growth.reactions.SQLS, model_growth.reactions.DXPS, model_growth.reactions.IPDDI,
     model_growth.reactions.GRTT]
fva_growth = flux_variability_analysis(model_growth, reaction_list = reactions_growth, fraction_of_optimum=1)
print (fva_growth)

        minimum   maximum
SQLS   0.000000  0.000000
DXPS   0.010663  0.010663
IPDDI -0.007996  0.002667
GRTT   0.002667  0.002667


In [ ]:

#quadratic_objective = modelQO.problem.Objective(modelQO.reactions.get_by_id('EX_sql_e').flux_expression* modelQO.reactions.get_by_id('BIOMASS_Ec_SynAuto').flux_expression,direction='max')
#modelQO.objective = quadratic_objective
#solution = modelQO.optimize(objective_sense=None)

In [ ]:
modelQO.summary()